# HRRR → WindNinja Wildfire Wind Downscaling Workflow

This notebook demonstrates an end-to-end workflow for:
1. Fetching HRRR 3km wind data using Herbie
2. Visualizing the coarse wind field
3. Preparing data for WindNinja downscaling
4. Running WindNinja to get terrain-adjusted 30-100m winds
5. Visualizing the high-resolution output

## Background

**HRRR (High-Resolution Rapid Refresh)**:
- 3km horizontal resolution
- Hourly updates (most frequent operational NWP model)
- CONUS coverage
- Best operational model for fire weather applications

**WindNinja**:
- Developed by USFS Missoula Fire Sciences Lab
- Downscales NWP winds to 30-100m using mass-conserving CFD
- Accounts for terrain effects (ridges, valleys, flow channeling)
- Critical for fire behavior prediction in complex terrain

## Setup

In [ ]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from fetch_hrrr import HRRRWindFetcher
from windninja_runner import WindNinjaConfig, WindNinjaRunner, generate_slurm_script

# For nice inline plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

## 1. Define Region of Interest

We'll focus on the Sierra Nevada foothills - a fire-prone region with complex terrain.

In [ ]:
# Paradise/Camp Fire region (November 2018)
REGION_NAME = "Paradise, CA (Sierra Foothills)"
BOUNDS = (-122.0, 39.5, -121.0, 40.0)  # (west, south, east, north)

# Alternative regions you might try:
# LA Basin / Santa Ana wind corridor
# LA_BOUNDS = (-118.5, 33.8, -117.5, 34.5)

# Wine Country (Tubbs Fire region)
# WINE_COUNTRY_BOUNDS = (-123.0, 38.2, -122.2, 38.8)

# Diablo wind corridor (East Bay hills)
# DIABLO_BOUNDS = (-122.3, 37.7, -121.8, 37.95)

print(f"Region: {REGION_NAME}")
print(f"Bounds: {BOUNDS}")
print(f"Approximate size: {abs(BOUNDS[2]-BOUNDS[0])*111:.0f} x {abs(BOUNDS[3]-BOUNDS[1])*111:.0f} km")

## 2. Fetch HRRR Surface Winds

In [ ]:
fetcher = HRRRWindFetcher(cache_dir=Path("../hrrr_cache"))

# Fetch most recent available HRRR analysis (typically 2 hours behind real-time)
fetch_time = datetime.utcnow().replace(minute=0, second=0, microsecond=0) - timedelta(hours=2)

print(f"Fetching HRRR for: {fetch_time} UTC")

ds = fetcher.fetch_surface_winds(
    date=fetch_time,
    forecast_hour=0,  # 0 = analysis, 1-48 = forecasts
    bounds=BOUNDS,
)

print("\n=== Dataset Info ===")
print(ds)

In [ ]:
# Examine the variables
print("Variables:")
for var in ds.data_vars:
    print(f"  {var}: {ds[var].attrs.get('long_name', 'N/A')} [{ds[var].attrs.get('units', 'N/A')}]")

print(f"\nGrid size: {ds.dims}")
print(f"Approximate resolution: {abs(ds.longitude[1].values - ds.longitude[0].values)*111:.1f} km")

## 3. Visualize HRRR Wind Field

In [ ]:
def plot_wind_field(ds, title="HRRR Surface Winds", skip=1):
    """
    Plot wind field with barbs or quiver arrows.
    
    Parameters
    ----------
    ds : xarray.Dataset
        Dataset with u10/si10, v10/wdir10 or wind_speed/wind_direction
    title : str
        Plot title
    skip : int
        Plot every Nth arrow (for readability)
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 7), 
                             subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Get wind components
    if 'u10' in ds:
        u = ds['u10'].values
        v = ds['v10'].values
    else:
        u = ds['u'].values
        v = ds['v'].values
    
    # Get wind speed
    if 'si10' in ds:
        speed = ds['si10'].values
    elif 'wind_speed' in ds:
        speed = ds['wind_speed'].values
    else:
        speed = np.sqrt(u**2 + v**2)
    
    # Get coordinates
    if 'longitude' in ds.coords:
        lon = ds['longitude'].values
        lat = ds['latitude'].values
    else:
        lon = ds['x'].values
        lat = ds['y'].values
    
    # Convert speed to mph for display
    speed_mph = speed * 2.237  # m/s to mph
    
    # Left panel: Wind speed contour
    ax1 = axes[0]
    cf = ax1.contourf(lon, lat, speed_mph, levels=np.arange(0, 35, 2.5),
                      cmap='YlOrRd', transform=ccrs.PlateCarree())
    ax1.add_feature(cfeature.STATES, linewidth=0.5)
    ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3, alpha=0.5)
    plt.colorbar(cf, ax=ax1, label='Wind Speed (mph)', shrink=0.7)
    ax1.set_title(f'{title}\nWind Speed')
    
    # Right panel: Wind vectors
    ax2 = axes[1]
    ax2.contourf(lon, lat, speed_mph, levels=np.arange(0, 35, 2.5),
                 cmap='YlOrRd', alpha=0.5, transform=ccrs.PlateCarree())
    
    # Quiver plot (subsample for readability)
    lon_grid, lat_grid = np.meshgrid(lon[::skip], lat[::skip])
    u_sub = u[::skip, ::skip]
    v_sub = v[::skip, ::skip]
    
    q = ax2.quiver(lon_grid, lat_grid, u_sub, v_sub,
                   transform=ccrs.PlateCarree(),
                   scale=150, width=0.003, headwidth=4)
    ax2.quiverkey(q, 0.85, 0.02, 10, '10 m/s', labelpos='W')
    
    ax2.add_feature(cfeature.STATES, linewidth=0.5)
    ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3, alpha=0.5)
    ax2.set_title(f'{title}\nWind Vectors')
    
    plt.tight_layout()
    return fig

fig = plot_wind_field(ds, title=f"HRRR Surface Winds\n{fetch_time} UTC", skip=2)
plt.savefig('../output/hrrr_wind_field.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Fetch Multi-Level Winds (Optional)

For more advanced applications, we can look at winds at different pressure levels to understand vertical wind shear.

In [ ]:
# Fetch winds at multiple pressure levels
ds_levels = fetcher.fetch_multilevel_winds(
    date=fetch_time,
    forecast_hour=0,
    levels=[925, 850, 700],  # Near-surface, ~1.5km, ~3km AGL
    bounds=BOUNDS,
)

print(ds_levels)

In [ ]:
# Plot vertical wind profile at a point
center_lon = (BOUNDS[0] + BOUNDS[2]) / 2
center_lat = (BOUNDS[1] + BOUNDS[3]) / 2

fig, ax = plt.subplots(figsize=(8, 6))

levels = [925, 850, 700]
# Note: Actual level extraction depends on how cfgrib structures the data
# This is illustrative - adjust based on actual ds_levels structure

ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Pressure Level (hPa)')
ax.set_title(f'Vertical Wind Profile at ({center_lon:.2f}, {center_lat:.2f})')
ax.invert_yaxis()  # Pressure decreases with height
ax.grid(True, alpha=0.3)
plt.show()

## 5. Export for WindNinja

WindNinja can be initialized in multiple ways:
1. **Domain Average**: Single wind speed/direction applied across domain
2. **Point Initialization**: From weather station observations
3. **WxModel Initialization**: From NWP model (HRRR, NAM, etc.)

For WxModel initialization, we can either:
- Use the raw GRIB2 file directly (WindNinja reads HRRR native format)
- Export to ASCII grid format for domain-average initialization

In [ ]:
output_dir = Path("../output/windninja_input")
output_dir.mkdir(parents=True, exist_ok=True)

# Export to ASCII grid (for domain-average initialization)
speed_file, dir_file = fetcher.to_windninja_ascii(
    ds,
    output_dir=output_dir,
    filename_prefix=f"hrrr_{fetch_time.strftime('%Y%m%d_%H%M')}"
)

print(f"Speed file: {speed_file}")
print(f"Direction file: {dir_file}")

# Also save as NetCDF for archival
nc_file = fetcher.to_netcdf(
    ds,
    output_dir / f"hrrr_{fetch_time.strftime('%Y%m%d_%H%M')}.nc"
)
print(f"NetCDF file: {nc_file}")

## 6. Prepare DEM for WindNinja

WindNinja needs a Digital Elevation Model (DEM) for terrain-adjusting winds. Options:

1. **SRTM**: ~30m global (automatic download in WindNinja)
2. **3DEP**: 10m or 1m for US (USGS)
3. **ASTER GDEM**: ~30m global alternative

For HPC runs, it's better to pre-download the DEM.

In [ ]:
# Example: Download DEM using rioxarray and py3dep (optional)
# Uncomment if you have py3dep installed

# import py3dep
# 
# dem = py3dep.get_dem(BOUNDS, resolution=30)  # 30m resolution
# dem_path = output_dir / "dem_30m.tif"
# dem.rio.to_raster(dem_path)
# print(f"DEM saved to: {dem_path}")

# For now, we'll create a placeholder path
dem_path = output_dir / "dem_30m.tif"
print(f"DEM would be saved to: {dem_path}")
print("(Use py3dep or download from USGS 3DEP for actual DEM)")

## 7. Configure WindNinja Run

In [ ]:
# Option A: Domain-average initialization (simple, fast)
# Uses mean wind from HRRR as uniform input

# Calculate domain-average wind
if 'si10' in ds:
    mean_speed = float(ds['si10'].mean().values) * 2.237  # m/s to mph
    mean_dir = float(ds['wdir10'].mean().values)
else:
    mean_speed = float(ds['wind_speed'].mean().values) * 2.237
    mean_dir = float(ds['wind_direction'].mean().values)

print(f"Domain-average wind: {mean_speed:.1f} mph from {mean_dir:.0f}°")

config_domain_avg = WindNinjaConfig(
    elevation_file=dem_path,
    output_path=Path("../output/windninja_output"),
    initialization_method="domainAverageInitialization",
    input_speed=mean_speed,
    input_speed_units="mph",
    input_direction=mean_dir,
    mesh_resolution=100.0,  # 100m output resolution
    num_threads=8,
    vegetation="brush",  # brush, grass, or trees
    write_ascii_output=True,
    write_shapefile_output=True,
)

print("\nWindNinja CLI arguments:")
for arg in config_domain_avg.to_cli_args():
    print(f"  {arg}")

In [ ]:
# Option B: HRRR model initialization (uses full spatial variability)
# Requires the raw GRIB2 file path

# The Herbie library caches GRIB2 files - find the path
from herbie import Herbie
H = Herbie(fetch_time, model="hrrr", product="sfc", fxx=0)
grib_path = H.get_localFilePath("(?:UGRD|VGRD):10 m above ground")

print(f"HRRR GRIB2 file: {grib_path}")

config_hrrr = WindNinjaConfig(
    elevation_file=dem_path,
    output_path=Path("../output/windninja_output_hrrr"),
    initialization_method="wxModelInitialization",
    wx_model_type="HRRR",
    forecast_filename=Path(grib_path) if grib_path else None,
    mesh_resolution=100.0,
    num_threads=8,
    write_ascii_output=True,
    write_shapefile_output=True,
)

print("\nWindNinja HRRR initialization CLI arguments:")
for arg in config_hrrr.to_cli_args():
    print(f"  {arg}")

## 8. Generate SLURM Script for HPC

For large domains or batch processing, run on HPC.

In [ ]:
slurm_script = generate_slurm_script(
    config_domain_avg,
    container_path=Path("./windninja.sif"),
    job_name="windninja_paradise",
    partition="standard",
    cpus_per_task=8,
    time_limit="01:00:00",
    memory="16G",
)

print(slurm_script)

# Save the script
slurm_path = Path("../output/run_windninja.slurm")
slurm_path.write_text(slurm_script)
print(f"\nSaved to: {slurm_path}")

## 9. Run WindNinja Locally (if available)

If you have WindNinja installed locally or in a container:

In [ ]:
# Uncomment to run locally
# runner = WindNinjaRunner(container_path=Path("./windninja.sif"))
# result = runner.run(config_domain_avg)
# print(f"Output files: {result['output_files']}")

print("WindNinja run skipped (no container available)")
print("To run:")
print("  1. Pull container: apptainer pull windninja.sif docker://firelab/windninja:latest")
print("  2. Uncomment the code above")

## 10. Visualize WindNinja Output

After running WindNinja, load and visualize the high-resolution output.

In [ ]:
def load_ascii_grid(filepath):
    """
    Load ESRI ASCII grid file into xarray DataArray.
    """
    with open(filepath) as f:
        # Parse header
        header = {}
        for _ in range(6):
            line = f.readline().split()
            header[line[0].lower()] = float(line[1])
        
        # Read data
        data = np.loadtxt(f)
    
    # Create coordinates
    ncols = int(header['ncols'])
    nrows = int(header['nrows'])
    xll = header['xllcorner']
    yll = header['yllcorner']
    cellsize = header['cellsize']
    nodata = header.get('nodata_value', -9999)
    
    # Replace nodata with NaN
    data[data == nodata] = np.nan
    
    # Flip data (ASCII is stored top-to-bottom)
    data = np.flipud(data)
    
    # Create coordinate arrays
    x = np.arange(xll, xll + ncols * cellsize, cellsize)[:ncols]
    y = np.arange(yll, yll + nrows * cellsize, cellsize)[:nrows]
    
    da = xr.DataArray(
        data,
        dims=['y', 'x'],
        coords={'y': y, 'x': x}
    )
    return da

# Example: Load WindNinja output (when available)
# wn_speed = load_ascii_grid("../output/windninja_output/dem_100m_vel.asc")
# wn_dir = load_ascii_grid("../output/windninja_output/dem_100m_ang.asc")

In [ ]:
def compare_hrrr_windninja(hrrr_ds, wn_speed, wn_dir, title="HRRR vs WindNinja"):
    """
    Side-by-side comparison of HRRR and WindNinja wind fields.
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # HRRR (left)
    ax1 = axes[0]
    if 'si10' in hrrr_ds:
        speed = hrrr_ds['si10'].values * 2.237  # m/s to mph
    else:
        speed = hrrr_ds['wind_speed'].values * 2.237
    
    if 'longitude' in hrrr_ds.coords:
        lon, lat = hrrr_ds['longitude'].values, hrrr_ds['latitude'].values
    else:
        lon, lat = hrrr_ds['x'].values, hrrr_ds['y'].values
    
    cf1 = ax1.contourf(lon, lat, speed, levels=np.arange(0, 35, 2.5), cmap='YlOrRd')
    plt.colorbar(cf1, ax=ax1, label='Wind Speed (mph)')
    ax1.set_title(f'{title}\nHRRR (~3km)')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    
    # WindNinja (right)
    ax2 = axes[1]
    cf2 = ax2.contourf(wn_speed.x, wn_speed.y, wn_speed.values, 
                       levels=np.arange(0, 35, 2.5), cmap='YlOrRd')
    plt.colorbar(cf2, ax=ax2, label='Wind Speed (mph)')
    ax2.set_title(f'{title}\nWindNinja (~100m)')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    
    plt.tight_layout()
    return fig

# Uncomment when WindNinja output is available:
# fig = compare_hrrr_windninja(ds, wn_speed, wn_dir)
# plt.savefig('../output/hrrr_vs_windninja.png', dpi=150, bbox_inches='tight')

print("Comparison plot skipped (no WindNinja output yet)")

## 11. Time Series Animation (Advanced)

For forecasting applications, fetch a time series and animate.

In [ ]:
# Fetch 6-hour time series
# start_time = fetch_time
# end_time = fetch_time + timedelta(hours=6)
# 
# ds_timeseries = fetcher.fetch_time_series(
#     start_date=start_time,
#     end_date=end_time,
#     bounds=BOUNDS,
#     hours_interval=1,
# )
# 
# print(ds_timeseries)

print("Time series fetch skipped (can be slow)")
print("Uncomment the code above to fetch 6 hours of HRRR data")

## Summary

This notebook demonstrated:

1. **HRRR Data Access**: Using Herbie to fetch operational 3km wind data
2. **Visualization**: Wind speed contours and quiver plots with Cartopy
3. **Export**: Converting to WindNinja-compatible formats (ASCII, NetCDF)
4. **WindNinja Config**: Setting up both domain-average and model initialization
5. **HPC Integration**: Generating SLURM scripts for batch processing

### Next Steps

1. **Download real DEM**: Use `py3dep` or USGS 3DEP for actual elevation data
2. **Pull WindNinja container**: `apptainer pull windninja.sif docker://firelab/windninja:latest`
3. **Run WindNinja**: Either locally or on HPC
4. **Validation**: Compare WindNinja output against RAWS observations
5. **Integration**: Feed into fire behavior models (FARSITE, FlamMap, etc.)